#### Question 3 -- Strimko

Here is some template information for handing in your Strimko solution.

We will review and test your solution by
1.  Putting test code at the end of the notebook
1.  Restart kernel & run all

This notebook must run from beginning to end without error, and must not produce any code output.

In [1]:
# Do not change this cell -- you may assume that the search framework and client interface
# files will be in the same directory as this notebook

from searchFramework import aStarSearch
from searchClientInterface import WorldState

##### Strimko Problem Definition

Input
1.  Initial assignments -- an NxN array.  Every entry in the array must be a positive integer between 1 and N, or 0.  A positive value is the initial value for that row and column.  A zero means the (row, column) begins unassigned.  Initial assignments must respect the game rules: the same value must not appear in the same row, column, or chain
2.  Chains -- a list of length at most N.  Each entry in the list is a list of (row, column) tuples defining a chain (a set of positions that are adjacent, and must not contain the same value

Output
1.  Solution board:  an NxN array where every (row, column) is assigned, and the assignment must satisfy row, column, and chain constraints

In [2]:
#  All of your code for Strimko -- except the heuristic -- goes in this cell
#
# Function accepts:
#   initialAssignments and chains as defined above
#   strategy specifies the evaluator;  the string must be either "bfs", or "dfs"
#   verbose and limit -- passed to the search function
#
# returns:  (solutionBoard, stats)
#   solutionBoard as defined above 
#   stats is the stats tuple returned by aStarSearch
#

# Your function may assume that initialAssignments and chains argument are correct


import copy
from searchClientInterface import Problem
from searchClientInterface import BFSEvaluator, DFSEvaluator

class StrimkoWorldState(WorldState):
    def __init__(self, board, chains):
        self._board = board
        self._chains = chains
        self._N = len(board)
        self._assignments = {}
        
    def __str__(self):
        return "{" + str(self._board) + "}"
    
    def __eq__(self, other):
        if isinstance(other, StrimkoWorldState):
            return self._board == other._board
        
    def __hash__(self):
        return hash(str(self._board))
    
    def successors(self):
        nextCoordinates = self.chooseUnassignedLocation()
        if nextCoordinates == None:
            return []
        
        return self.allAssignmentsFor(nextCoordinates)
    
    def chooseUnassignedLocation(self):
        for x in range(self._N):
            for y in range(self._N):
                if self._board[x][y] == 0:
                    return (x, y)
        return None
    
    def allAssignmentsFor(self, nextCoordinates):
        visitedNumbers = set()
        for i in range(self._N):
            visitedNumbers.add(self._board[nextCoordinates[0]][i])
            visitedNumbers.add(self._board[i][nextCoordinates[1]])
        
        for chain in self._chains:
            if nextCoordinates in chain:
                for (x, y) in chain:
                    visitedNumbers.add(self._board[x][y])
            
        availableNumbers = set([i for i in range(1, (self._N + 1))]) - visitedNumbers
        candidates = [(self.assignNumber(nextCoordinates, num), (nextCoordinates, num)) for num in availableNumbers]
        return [c for c in candidates if c]
    
    def assignNumber(self, nextCoordinates, num):
        s = copy.deepcopy(self)
        (x, y) = nextCoordinates
        s._board[x][y] = num
        self._assignments[(x, y)] = num
        return s
    
    def isSolution(self):        
        for x in range(self._N):
            row = set()
            col = set()
            for y in range(self._N):
                rowVal = self._board[x][y]
                colVal = self._board[y][x]
                
                if rowVal < 1 or colVal < 1 or rowVal > self._N or colVal > self._N or (rowVal in row) or (colVal in col):
                    return False
                row.add(rowVal)
                col.add(colVal)
            
        for chain in self._chains:
            chainTracker = set()
            for pos in chain:
                if self._board[pos[0]][pos[1]] in chainTracker:
                    return False
                chainTracker.add(self._board[pos[0]][pos[1]])
        
        return True
        
class StrimkoProblem(Problem):
    def __init__(self, board, chains):
        self._state = StrimkoWorldState(board, chains)
        
    def initial(self):
        return self._state
    
    def isGoal(self, state):
        return state.isSolution()


def solveStrimko(initialAssignments, chains, strategy="bfs", verbose=None, limit=None):
    evaluator = BFSEvaluator() if strategy == "bfs" else DFSEvaluator()
    (actions, perf) = aStarSearch(StrimkoProblem(initialAssignments, chains), evaluator, verbose, limit)
    if not actions:
        return (actions, perf)
    
    finalAssignments = copy.deepcopy(initialAssignments)
    for action in actions:
        ((x, y), num) = action
        finalAssignments[x][y] = num
        
    return (finalAssignments, perf)


#### Choice of Search Strategy

In the markdown cell below, discuss the choice of breadth-first versus depth-first search.

Which is the preferable strategy?  Why, and run experiments to support your conclusion.

**Depth-first search is the preferred strategy**. (**Why**) BFS is suitable for searching solutions which are closer to the source, that is, it considers all neighbors first and there not suitable for decision making trees used in games or puzzles. Whereas DFS is more suitable when the solutions are away from the source. In this approach, we make a decision, then explore all paths through this decision. If this leads to a win situation, we then stop.
    
***Evidence/Experiments***: Following are the performance stats (bfs vs dfs) for different boards given in the assignment. You will observe the time taken to find a solution using DFS is much faster than BFS. The number of nodes visited using DFS is also much smaller compared to BFS especially as the complexity of input increases.
    

1. **4x4 board given in the question**: As you can see below the time taken for ***BFS (1.94 secs) is twice that of DFS (0.99 secs)***.     

   * **BFS**: `([[5, 1, 2, 3, 4], [3, 5, 4, 2, 1], [4, 2, 5, 1, 3], [1, 4, 3, 5, 2], [2, 3, 1, 4, 5]], (1.9409410000000094, 7575, 0, 1152))`
    
   * **DFS**: `([[5, 1, 2, 3, 4], [3, 5, 4, 2, 1], [4, 2, 5, 1, 3], [1, 4, 3, 5, 2], [2, 3, 1, 4, 5]], (0.9970129999999813, 5229, 0, 15))`
    
    
2. Following are stats from **test boards**:

  a. **3x3 (board3_a)**: BFS (0.0014 secs) vs DFS (0.0008 secs)
   
   * **BFS**: `([[3, 2, 1], [1, 3, 2], [2, 1, 3]], (0.0014080000000262771, 7, 0, 1))`
   * **DFS**: `([[3, 2, 1], [1, 3, 2], [2, 1, 3]], (0.0008359999999925094, 7, 0, 1))`
        
  b. **4x4 (board4_b)**: BFS (0.00277 secs) vs DFS (0.0022 secs)
    
   * **BFS**: `([[3, 1, 4, 2], [2, 4, 1, 3], [1, 3, 2, 4], [4, 2, 3, 1]], (0.002774000000044907, 18, 0, 2))`
   * **DFS**: `([[3, 1, 4, 2], [2, 4, 1, 3], [1, 3, 2, 4], [4, 2, 3, 1]], (0.002284999999972115, 14, 0, 3))`
        
  c. **5x5 (board5_a)**: BFS (0.00338 secs) vs DFS (0.00295 secs)
    
   * **BFS**: `([[4, 1, 2, 3, 5], [5, 3, 4, 1, 2], [3, 4, 5, 2, 1], [2, 5, 1, 4, 3], [1, 2, 3, 5, 4]], (0.0033849999999802094, 21, 0, 3))`
   * **DFS**: `([[4, 1, 2, 3, 5], [5, 3, 4, 1, 2], [3, 4, 5, 2, 1], [2, 5, 1, 4, 3], [1, 2, 3, 5, 4]], (0.002957999999978256, 18, 0, 3))`
        
  d. **5x5 (board5_b)**: BFS (0.0036 secs) vs DFS (0.003 secs)
    
   * **BFS**: `([[5, 2, 1, 3, 4], [3, 5, 4, 1, 2], [4, 1, 5, 2, 3], [2, 4, 3, 5, 1], [1, 3, 2, 4, 5]], (0.0036059999999906722, 20, 0, 3))`
   * **DFS**: `([[5, 2, 1, 3, 4], [3, 5, 4, 1, 2], [4, 1, 5, 2, 3], [2, 4, 3, 5, 1], [1, 3, 2, 4, 5]], (0.0030869999999936226, 20, 0, 3))`

